# Applied Data Science Capstone Course by IBM on Coursera

## Peer-graded Assignment: Capstone Project - The Battle of Neighborhoods 

### Opening a New Shopping Mall in Mumbai, India(Financial Capital of India)

* Build a dataframe of neighborhoods in Mumbai, India by web scraping the data from Wikipedia page
* Get the geographical coordinates of the neighborhoods using geocoder
* Obtain the venue data for the neighborhoods using Foursquare API
* Explore and cluster the neighborhoods
* Select the best cluster to open a new shopping mall

__Import Libraries__

In [8]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


__Scrap the data Scrap data from Wikipedia page into a DataFrame__

In [9]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Mumbai").text

In [13]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')
# create a list to store neighborhood data
neighborhoodList = []


In [14]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [15]:
# create a new DataFrame from the list
Mumbai_df = pd.DataFrame({"Neighborhood": neighborhoodList})

Mumbai_df.head()

,Neighborhood
0,List of neighbourhoods in Mumbai
1,Aarey Milk Colony
2,Agripada
3,Altamount Road
4,"Amboli, Mumbai"


In [17]:
Mumbai_df.drop(Mumbai_df.index[0])


,Neighborhood
1,Aarey Milk Colony
2,Agripada
3,Altamount Road
4,"Amboli, Mumbai"
5,Amrut Nagar
6,Antop Hill
7,Anushakti Nagar
8,Asalfa
9,Badhwar Park
10,Baiganwadi


In [18]:
# print the number of rows of the dataframe

Mumbai_df.shape

(137, 1)

__Get the geographical coordinates using geocoder__

In [22]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Pune, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [23]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in Mumbai_df["Neighborhood"].tolist() ]

In [24]:
coords



[[19.19590820260835, 72.87832817277396],
 [30.76407000000006, 76.75047000000006],
 [18.97628000000003, 72.82615000000004],
 [13.062170000000037, 80.26308000000006],
 [19.129060000000038, 72.84644000000003],
 [18.471690000000024, 73.82297000000005],
 [19.026140000000055, 72.86645000000004],
 [18.52824000000004, 73.90464000000003],
 [19.095250000000078, 72.89255000000003],
 [18.51692006684779, 73.8562815],
 [19.008560000000045, 73.66081000000008],
 [18.935410000000047, 72.83979000000005],
 [18.599946873160604, 73.73583355176294],
 [19.062080000000037, 72.86844000000008],
 [19.167850000000044, 72.83292000000006],
 [18.662073485429048, 73.8001654365803],
 [18.126330000000053, 74.74283000000008],
 [18.969030000000032, 72.80384000000004],
 [18.980740000000026, 72.84072000000003],
 [18.493099208436732, 73.85754421357262],
 [18.53707000000003, 73.42457000000007],
 [18.996790000000033, 72.84654000000006],
 [19.011310000000037, 73.97475000000003],
 [18.81242000000003, 74.15602000000007],
 [18.94

In [25]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [28]:
# merge the coordinates into the original dataframe
Mumbai_df['Latitude'] = df_coords['Latitude']
Mumbai_df['Longitude'] = df_coords['Longitude']

In [29]:
# check the neighborhoods and the coordinates
print(Mumbai_df.shape)
Mumbai_df

(137, 3)


,Neighborhood,Latitude,Longitude
0,List of neighbourhoods in Mumbai,19.195908,72.878328
1,Aarey Milk Colony,30.764070,76.750470
2,Agripada,18.976280,72.826150
3,Altamount Road,13.062170,80.263080
4,"Amboli, Mumbai",19.129060,72.846440
5,Amrut Nagar,18.471690,73.822970
6,Antop Hill,19.026140,72.866450
7,Anushakti Nagar,18.528240,73.904640
8,Asalfa,19.095250,72.892550
9,Badhwar Park,18.516920,73.856281


In [30]:
Mumbai_df.drop(Mumbai_df.index[0])

,Neighborhood,Latitude,Longitude
1,Aarey Milk Colony,30.764070,76.750470
2,Agripada,18.976280,72.826150
3,Altamount Road,13.062170,80.263080
4,"Amboli, Mumbai",19.129060,72.846440
5,Amrut Nagar,18.471690,73.822970
6,Antop Hill,19.026140,72.866450
7,Anushakti Nagar,18.528240,73.904640
8,Asalfa,19.095250,72.892550
9,Badhwar Park,18.516920,73.856281
10,Baiganwadi,19.008560,73.660810


In [32]:
# save the DataFrame as CSV file
Mumbai_df.to_csv("Mumbai_df.csv", index=False)

__Create a map of Mumbai with neighborhoods superimposed on top__

In [33]:
 #get the coordinates of Mumbai
address = 'Mumbai, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai, India 18.9387711, 72.8353355.


In [35]:
# create map of Mumbai using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(Mumbai_df['Latitude'], Mumbai_df['Longitude'], Mumbai_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [36]:
# save the map as HTML file
map_kl.save('map_kl.html')

__Use the Foursquare API to explore the neighborhoods__

In [37]:
# define Foursquare Credentials and Version
CLIENT_ID = '3GJUQFOI1XUVDIAPEGDLS54WNSFGTRLH0FUW4SAI15MIOCA2' # your Foursquare ID
CLIENT_SECRET = 'FBEMQEDBZKLY4BJMBPSC0WUHZIAPNOQ224DEVN0L3USYGD2M' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3GJUQFOI1XUVDIAPEGDLS54WNSFGTRLH0FUW4SAI15MIOCA2
CLIENT_SECRET:FBEMQEDBZKLY4BJMBPSC0WUHZIAPNOQ224DEVN0L3USYGD2M


__Now, let's get the top 100 venues that are within a radius of 2000 meters.__

In [39]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(Mumbai_df['Latitude'], Mumbai_df['Longitude'], Mumbai_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [41]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.tail(10)

(7640, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
7630,Zaveri Bazaar,18.95028,72.83157,Café Universal,18.936021,72.837453,Irani Cafe
7631,Zaveri Bazaar,18.95028,72.83157,Zam Zam Restaurant,18.956441,72.832046,Indian Restaurant
7632,Zaveri Bazaar,18.95028,72.83157,Noorani Milk Centre,18.954749,72.833382,Café
7633,Zaveri Bazaar,18.95028,72.83157,Manish Market,18.947704,72.835407,Electronics Store
7634,Zaveri Bazaar,18.95028,72.83157,5 Spice,18.943540,72.825159,Chinese Restaurant
7635,Zaveri Bazaar,18.95028,72.83157,5 Spice,18.933491,72.835955,Chinese Restaurant
7636,Zaveri Bazaar,18.95028,72.83157,Mad Over Donuts,18.935131,72.826910,Donut Shop
7637,Zaveri Bazaar,18.95028,72.83157,Apoorva - The seafood capital,18.933135,72.834527,Seafood Restaurant
7638,Zaveri Bazaar,18.95028,72.83157,Lamington Road,18.960566,72.817226,Electronics Store
7639,Zaveri Bazaar,18.95028,72.83157,McDonald's,18.938985,72.834504,Fast Food Restaurant


__Let's check how many venues were returned for each neighorhood__

In [42]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Aarey Milk Colony,5,5,5,5,5,5
Agripada,75,75,75,75,75,75
Altamount Road,100,100,100,100,100,100
"Amboli, Mumbai",75,75,75,75,75,75
Amrut Nagar,21,21,21,21,21,21
Antop Hill,81,81,81,81,81,81
Anushakti Nagar,100,100,100,100,100,100
Asalfa,100,100,100,100,100,100
Badhwar Park,100,100,100,100,100,100


__Let's find out how many unique categories can be curated from all the returned venues__

In [43]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 255 uniques categories.


In [45]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Neighborhood', 'Pizza Place', 'Fast Food Restaurant',
       'Indian Restaurant', 'Gym', 'Coffee Shop', 'Restaurant',
       'Ice Cream Shop', 'Nightclub', 'Sandwich Place',
       'General Entertainment', 'Department Store', 'Stadium',
       'Dessert Shop', 'Snack Place', 'Miscellaneous Shop', 'Smoke Shop',
       'Food Truck', 'Residential Building (Apartment / Condo)',
       'Volleyball Court', 'Indie Movie Theater', 'Juice Bar',
       'Breakfast Spot', 'Steakhouse', 'Playground', 'Shoe Store',
       'Food Court', 'Hostel', 'Rental Car Location', 'Club House',
       'Bakery', 'Golf Course', 'History Museum',
       'Middle Eastern Restaurant', 'Music Venue', 'Scenic Lookout',
       'Spa', 'Bar', 'Chinese Restaurant', 'Hotel', 'Deli / Bodega',
       'Asian Restaurant', 'Bengali Restaurant', 'Café', 'Cupcake Shop',
       'Planetarium', 'BBQ Joint', 'Art Gallery', 'Italian Restaurant',
       'Clothing Store'], dtype=object)

In [46]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

__Analyze Each Neighborhood__

In [49]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.tail()

(7640, 256)


,Neighborhoods,ATM,Accessories Store,African Restaurant,Airport,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Beach Bar,Bed & Breakfast,Beer Garden,Bengali Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Camera Store,Candy Store,Chaat Place,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Donut Shop,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Electronics Store,English Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Goan Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hyderabadi Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Indoor Play Area,Intersection,Irani Cafe,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoors & Recreation,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pool Hall,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,River,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Track,Track Stadium,Trail,Train,Train Station,Vegetarian / Vegan Restaurant,Volleyball Court,Watch Shop,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
7635,Zaveri Bazaar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

__Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category__

In [50]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(126, 256)


,Neighborhoods,ATM,Accessories Store,African Restaurant,Airport,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Beach Bar,Bed & Breakfast,Beer Garden,Bengali Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Camera Store,Candy Store,Chaat Place,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Donut Shop,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Electronics Store,English Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Goan Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hyderabadi Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Indoor Play Area,Intersection,Irani Cafe,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoors & Recreation,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pool Hall,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,River,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Track,Track Stadium,Trail,Train,Train Station,Vegetarian / Vegan Restaurant,Volleyball Court,Watch Shop,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,Aarey Milk Colony,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,

In [51]:
len(kl_grouped[kl_grouped["Shopping Mall"] > 0])

36

__Create a new DataFrame for Shopping Mall data only__

In [52]:
kl_mall = kl_grouped[["Neighborhoods","Shopping Mall"]]

In [53]:
kl_mall.head()

,Neighborhoods,Shopping Mall
0,Aarey Milk Colony,0.000000
1,Agripada,0.000000
2,Altamount Road,0.010000
3,"Amboli, Mumbai",0.013333
4,Amrut Nagar,0.047619


# Cluster Neighborhoods

Run k-means to cluster the neighborhoods in Mumbai into 3 clusters.

In [54]:
# set number of clusters
kclusters = 3

kl_clustering = kl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 2, 2, 1, 2, 0, 2, 0, 0], dtype=int32)

In [55]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [56]:


kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()



,Neighborhood,Shopping Mall,Cluster Labels
0,Aarey Milk Colony,0.000000,0
1,Agripada,0.000000,0
2,Altamount Road,0.010000,2
3,"Amboli, Mumbai",0.013333,2
4,Amrut Nagar,0.047619,1


In [59]:
# merge pune_grouped with Pune_data to add latitude/longitude for each neighborhood
kl_merged = kl_merged.join(Mumbai_df.set_index("Neighborhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(126, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Aarey Milk Colony,0.000000,0,30.76407,76.75047
1,Agripada,0.000000,0,18.97628,72.82615
2,Altamount Road,0.010000,2,13.06217,80.26308
3,"Amboli, Mumbai",0.013333,2,19.12906,72.84644
4,Amrut Nagar,0.047619,1,18.47169,73.82297


__sort the results by Cluster Labels as 0,1,2__

In [60]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(126, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Aarey Milk Colony,0.000000,0,30.764070,76.750470
84,Marine Lines,0.000000,0,18.943430,72.823190
83,"Marine Drive, Mumbai",0.000000,0,18.946660,72.824560
81,"Mandvi, Mumbai",0.000000,0,19.047906,72.921587
80,Malabar Hill,0.000000,0,18.950000,72.783330
76,"Mahalaxmi, Mumbai",0.000000,0,19.043384,72.872141
75,Madh Island,0.000000,0,19.146810,72.790670
74,Lokhandwala Complex,0.000000,0,21.775140,78.260380
73,Lohar Chawl,0.000000,0,18.947030,72.832150
72,List of neighbourhoods in Mumbai,0.000000,0,19.195908,72.878328


### Finally, let's visualize the resulting clusters

In [61]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [62]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

## Examine Clusters

__Cluster 0__

In [63]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Aarey Milk Colony,0.0,0,30.764070,76.750470
84,Marine Lines,0.0,0,18.943430,72.823190
83,"Marine Drive, Mumbai",0.0,0,18.946660,72.824560
81,"Mandvi, Mumbai",0.0,0,19.047906,72.921587
80,Malabar Hill,0.0,0,18.950000,72.783330
76,"Mahalaxmi, Mumbai",0.0,0,19.043384,72.872141
75,Madh Island,0.0,0,19.146810,72.790670
74,Lokhandwala Complex,0.0,0,21.775140,78.260380
73,Lohar Chawl,0.0,0,18.947030,72.832150
72,List of neighbourhoods in Mumbai,0.0,0,19.195908,72.878328


In [70]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0].count()

Neighborhood      90
Shopping Mall     90
Cluster Labels    90
Latitude          90
Longitude         90
dtype: int64

__Cluster 1__

In [64]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
12,Bangur Nagar,0.030000,1,19.167850,72.832920
32,Dava Bazaar,0.026316,1,26.292190,73.013590
24,Colaba,0.041667,1,18.906940,72.810560
4,Amrut Nagar,0.047619,1,18.471690,73.822970
94,Navy Nagar,0.031250,1,18.906030,72.815450
96,Null Bazaar,0.033333,1,18.531250,73.811140
56,Kajuwadi,0.030000,1,19.198220,72.961710
16,C.G.S. colony,0.031746,1,18.493099,73.857544
55,Jagruti Nagar,0.034483,1,18.487501,73.860643


In [71]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1].count()

Neighborhood      9
Shopping Mall     9
Cluster Labels    9
Latitude          9
Longitude         9
dtype: int64

__Cluster 2__

In [65]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
78,Mahim,0.020833,2,18.574079,73.780691
77,"Maheshwari Udyan, Mumbai",0.010000,2,19.027600,72.855220
117,Thakkar Bappa Colony,0.011905,2,19.059600,72.887890
111,"Shastri Nagar, Goregaon",0.020619,2,19.160450,72.836510
5,Antop Hill,0.012346,2,19.026140,72.866450
119,"Versova, Mumbai",0.010000,2,19.137690,72.813480
3,"Amboli, Mumbai",0.013333,2,19.129060,72.846440
2,Altamount Road,0.010000,2,13.062170,80.263080
7,Asalfa,0.010000,2,19.095250,72.892550
47,Gorai,0.010870,2,18.564260,73.801220


In [72]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2].count()

Neighborhood      27
Shopping Mall     27
Cluster Labels    27
Latitude          27
Longitude         27
dtype: int64

### Observations

After performing machine learning by clustering the data into 3 clusters based on their similarities,     
and lastly providing recommendations to the relevant stakeholders i.e. property developers and investors  
regarding the best locations to open a new shopping mall. To answer the business question that was raised   
in the introduction section, the answer proposed by this project is: The neighbourhoods in cluster 1 are  
the most preferred locations to open a new shopping mall. The findings of this project will help the  
relevant stakeholders to capitalize on the opportunities on high potential locations while avoiding overcrowded  
areas in their decisions to open a new shopping mall.   